In [1]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
from pykrx import stock
import OpenDartReader
import csv

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.simplefilter(action ='ignore',category=FutureWarning)

In [2]:
f = open('dartApiKey.cfg', 'r',-1, "utf-8")
api_key = f.readline()
dart = OpenDartReader(api_key)

In [36]:
def Graph(company,From, To) :
    df_info = {}
    
    for year in range(From,To+1):
        #재무제표 전체 Read
        fs = dart.finstate_all(company,year) 
        
        #당기
        def fs_nm(val) : 
            fs_nm = int(fs[fs['account_nm'].str.contains(val)][['thstrm_amount']].iloc[0,0].replace(',',''))
            return fs_nm
        
        def fs_nm_last(val) :
            fs_nm_last = int(fs[fs['account_nm'].str.contains(val)][['frmtrm_amount']].iloc[0,0].replace(',',''))
            return fs_nm_last
        
        #ADPS
        
        dv = dart.report(company,'배당',year)
        dv_dps= dv[dv['se'] == '주당 현금배당금(원)']
        dv_eps= dv[dv['se'].str.contains('주당순이익')]
        dv_yield=dv[dv['se'].str.contains('현금배당수익률')]
        dv_TD = dv[dv['se'].str.contains('현금배당금총액')]
        
        #factors
        
        equity = fs_nm('자본총계')
        equity_last = fs_nm_last('자본총계')
        
        liability = fs_nm('부채총계')
        liability_last = fs_nm_last('부채총계')
        
        current_liabilities = fs_nm('유동부채')
        non_current_liabilities = liability - current_liabilities # 비유동부채
        
        assets = equity + liability #당기자산
        assets_last = equity_last + liability_last #전기자산
        
        revenue = fs_nm('매출액|영업수익')
        revenue_last = fs_nm_last('매출액|영업수익')
        
        income = fs_nm('영업이익')
        income_last = fs_nm_last('영업이익')
        
        profit=fs_nm('당기순이익|반기순이익|기순이익') #당기 순이익
        profit_last = fs_nm_last('당기순이익|반기순이익|기순이익') #전기 순이익
        
        try:
            CostOfSales = abs(fs_nm('매출원가|재료비'))
            CostOfMgnt = abs(fs_nm('판매비와관리비|영업비용'))
            
            GPM = round((revenue-CostOfSales)/revenue*100,1) #매출총이익률
        except :
            CostOfSales = 0
            CostOfMgnt = abs(fs_nm('판매비와관리비|영업비용'))
            GPM = 0
        
        #주당순이익
        EPS = int(dv_eps[['thstrm']].iloc[0,0].replace(',','').strip()) 
        
        
        ##
        try:
            #주당 현금배당
            DPS = int(dv_dps[['thstrm']].iloc[0,0].replace(',','').strip*())
            Yield =round(int(dv_yield[['thstrm']].iloc[0,0].replace('.','').strip())/100,1)
            TD = int(dv_TD[['thstrm']].iloc[0,0].replace(',','').strip())*1000000
            
        except:
            DPS = 0
            Yield =0
            TD=0
        
        ##
        CF_operating = fs_nm('영업활동') #영업활동현금흐름
        CF_investing = fs_nm('투자활동') #투자활동현금흐름
        CF_financing = fs_nm('재무활동') #재무활동현금흐름
        
        ##
        try :
            CAPEX = fs_nm('유형자산의 취득') + fs_nm('무형자산의 취득') #유형자산취득 + 무형자산취득
        except :
            CAPEX = 0
            
            
        ## 재고자산
        try :
            inventory_assets = fs_nm('재고자산')#당기 재고자산
            inventory_assets_last = fs_nm_last('재고자산')#전기 재고자산
            #재고자산 회전율
            turnover_inventory = round(revenue / ((inventory_assets + inventory_assets_last)/2),2)
        except :
            inventory_assets = 0
            inventory_assets_last = 0
            turnover_inventory = 0
            
        turnover_assets = round(revenue / ((assets + assets_last)/2),2) #총자산 회전율
        
        
        ## Main Index
        
        #잉여현금흐름
        FCF = CF_operating = CAPEX # 영업활동현금흐름 - (유형+무형)자산취득
        #자기자본 수익률
        ROE = round(profit / ((equity + equity_last)/2)*100,1)  # 자기자본수익률
        #자산수익률
        ROA = round(profit/((assets + assets_last)/2) *100,1)
        #영업이익률 
        OPM = round(income / revenue * 100,1)
        #순이익률
        NPM = round(profit/revenue * 100 ,1)
        #배당성향
        PR = round(DPS/EPS * 100,1)
        #부채비율
        DR = round(liability / equity * 100, 1)
        
        #유동부채 비율
        DR_current=round(current_liabilities/equity * 100,1)
        
        #Growth Rate_revenue
        GR_revenue=round((revenue/revenue_last-1)*100,1) #매출액 증가율
        
        #Growth Rate_income
        GR_income = round((income/income_last-1)*100,1) #영업이익  증가율
        
        #Growth Rate_profit
        GR_profit = round((profit/profit_last)-1 *100,1) #순이익 증가율
        
        
        #dictionary에 담기
        
        df_info[str(year)]=\
        revenue,CostOfSales,CostOfMgnt, income,profit,\
        GPM,OPM,NPM,ROE,ROA,GR_revenue,GR_income,GR_profit,\
        CF_operating, CAPEX, FCF, TD, CF_investing, CF_financing,\
        EPS,DPS,PR,Yield,\
        DR,DR_current,\
        turnover_assets,turnover_inventory,\
        inventory_assets, assets, equity, liability
        
        Index = ['매출액','매출원가','판관비','영업이익','순이익',
                'GPM','OPM','NPM',
                'ROE','ROA',
                '매출 증감률','영업이익 증감률','순이익 증감률',
                '영업활동현금흐름','CAPEX','FCF','배당금 총액','투자활동현금흐름','재무활동현금흐름',
                'EPS','DPS','배당성향','배당률',
                '부채비율','유동부채비율',
                '자산회전율','재고자산회전율',
                '재고자산','자산','자본','부채']
        
        df=pd.DataFrame(data= df_info,index=Index)
        
        
        
        ###그래프 그리기 ###
        
        #종목명
        company_nm = dart.company(company)['stock_name']
        #회사 영문명
        company_nm_eng = dart.company(company)['corp_name_eng']
        #종목코드
        company_stock_code = dart.company(company)['stock_code']
        
        title = company_nm + '(' + company_nm_eng + ', ' + company_stock_code + ')'
        titles = dict(text = title,x=0.5,y=0.85)
        
        #colors
        marker_colors = ['rgb(27,38,81)','rgb(205,32,40)','rgb(22,108,150)','rgb(255,255,255)', 'rgb(237,234,225)']
        
        # copyright
        annotations = []
        annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1, xanchor='center', yanchor='top',
                            text='Copyright 2022. Xakyh. All rights reserved.',
                            font=dict(family='Arial', size=12, color='rgb(150,150,150)'),
                            showarrow=False))
    
        x_data = df.keys() # x축
    
    
         # Combo chart
        fig = make_subplots(specs=[[{'secondary_y': True}]])
        y_data_bar = ['매출액', '매출원가', '판관비']
        y_data_line = ['영업이익', '순이익']

        for y_data, color in zip(y_data_bar, marker_colors) : 
            fig.add_trace(go.Bar(name = y_data, x = x_data, y = df.loc[y_data],
                                 text = df.loc[y_data], textposition = 'outside', width = 0.2, marker_color = color), 
                                 secondary_y = False)
        for y_data, color in zip(y_data_line, marker_colors) : 
            fig.add_trace(go.Scatter(mode='lines+markers+text',
                                     name = y_data, x = x_data, y= df.loc[y_data], 
                                     text = df.loc[y_data], textposition = 'top center', marker_color= color), 
                                     secondary_y = True)   

        fig.update_traces(texttemplate='%{text:.3s}')
        fig.update_yaxes(range=[0, max(df.loc[y_data_bar[0]]) * 2 ], secondary_y = False)
        fig.update_yaxes(range=[-max(df.loc[y_data_line[0]]) , max(df.loc[y_data_line[0]]) * 1.2 ], secondary_y = True)
        fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)
        fig.update_layout(title = titles, titlefont_size=15, legend_title_text='(단위 : 원)',
                          xaxis_tickformat = 'd', annotations=annotations)
        fig.show()

        # Line chart
        fig = make_subplots(specs=[[{'secondary_y': True}]])
        y_datas = ['GPM', 'OPM', 'NPM']

        for y_data, color in zip(y_datas, marker_colors) : 
            fig.add_trace(go.Scatter(mode='lines+markers+text', name = y_data, x = x_data, y= df.loc[y_data], 
                                     text = df.loc[y_data], textposition = 'top center', marker_color= color), 
                                     secondary_y = True)                  

        fig.update_traces(texttemplate='%{text:.3s}')
        fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)
        fig.update_xaxes(showgrid=False, zeroline=False)
        fig.update_layout(title = titles, titlefont_size=15, legend_title_text='(단위 : %)',
                          xaxis_tickformat = 'd', annotations=annotations)
        fig.show()  

        # Bar chart
        fig = make_subplots(specs=[[{'secondary_y': True}]])
        y_datas = ['매출 증감률', '영업이익 증감률', '순이익 증감률']

        for y_data, color in zip(y_datas, marker_colors) : 
            fig.add_trace(go.Bar(name = y_data, x = x_data, y = df.loc[y_data],
                                 text = df.loc[y_data], textposition = 'outside',width = 0.2, marker_color = color), 
                                 secondary_y = False)

        fig.update_traces(texttemplate='%{text:.3s}')
        fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)
        fig.update_xaxes(showgrid=False, zeroline=False)
        fig.update_layout(title = titles, titlefont_size=15, legend_title_text='(단위 : %)',
                          xaxis_tickformat = 'd', annotations=annotations)
        fig.show()

In [ ]:
def GreatStratig(company,tick,year)
#재무제표 전체 Read
        fs = dart.finstate_all(company,year) 
        
        #당기
        def fs_nm(val) : 
            fs_nm = int(fs[fs['account_nm'].str.contains(val)][['thstrm_amount']].iloc[0,0].replace(',',''))
            return fs_nm
        
        def fs_nm_last(val) :
            fs_nm_last = int(fs[fs['account_nm'].str.contains(val)][['frmtrm_amount']].iloc[0,0].replace(',',''))
            return fs_nm_last
        
        #For PER
        income = fs_nm('영업이익')
        income_last = fs_nm_last('영업이익')
        
        #For PBR
        equity = fs_nm('자본총계')
        equity_last = fs_nm_last('자본총계')
        
        #For PCR
        CF_operating = fs_nm('영업활동')
        
        #For PSR
        revenue = fs_nm('매출액|영업수익')
        revenue_last = fs_nm_last('매출액|영업수익')
        
        
        
        #Growth Rate_revenue
        GR_revenue=round((revenue/revenue_last-1)*100,1) #매출액 증가율
        
        #Growth Rate_income
        GR_income = round((income/income_last-1)*100,1) #영업이익  증가율
        
        #Growth Rate_profit
        GR_profit = round((profit/profit_last)-1 *100,1) #순이익 증가율
        
        
        
        ret=[company, ]
        

In [15]:
fcsv = open('data_5346_20220214.csv','r')
rdr = csv.reader(fcsv)
rdr

for line in rdr:
    cName = line[1]
    if(cName != '종목명'):
        
    print(line[1])

종목명
삼성전자
SK하이닉스
NAVER
삼성바이오로직스
LG화학
카카오
현대차
삼성SDI
기아
KB금융
POSCO
셀트리온
현대모비스
카카오뱅크
신한지주
LG전자
삼성물산
SK이노베이션
카카오페이
SK
하나금융지주
LG생활건강
한국전력
크래프톤
삼성생명
SK바이오사이언스
HMM
삼성전기
SK텔레콤
LG
엔씨소프트
KT&G
우리금융지주
삼성에스디에스
대한항공
하이브
S-Oil
아모레퍼시픽
삼성화재
고려아연
현대중공업
넷마블
포스코케미칼
SK아이이테크놀로지
KT
기업은행
두산중공업
SK스퀘어
LG이노텍
롯데케미칼
LG디스플레이
현대글로비스
SK바이오팜
한화솔루션
한온시스템
LG유플러스
한국조선해양
미래에셋증권
메리츠금융지주
CJ제일제당
강원랜드
현대제철
코웨이
SKC
금호석유
삼성중공업
DB손해보험
현대건설
삼성엔지니어링
한국금융지주
한국타이어앤테크놀로지
유한양행
메리츠증권
일진머티리얼즈
오리온
쌍용C&E
두산밥캣
삼성증권
GS
삼성카드
한진칼
현대중공업지주
아모레G
이마트
한국항공우주
NH투자증권
KCC
GS건설
한국가스공사
팬오션
한미약품
호텔신라
한미사이언스
DB하이텍
롯데지주
한화시스템
한전기술
한화생명
BNK금융지주
GS리테일
BGF리테일
현대미포조선
제일기획
동서
CJ대한통운
에스원
키움증권
신세계
하이트진로
CJ
현대해상
한화에어로스페이스
한솔케미칼
롯데쇼핑
포스코인터내셔널
대우조선해양
대우건설
DL이앤씨
두산퓨얼셀
SK케미칼
한화
농심
녹십자
만도
현대로템
영원무역
씨에스윈드
OCI
롯데정밀화학
휠라홀딩스
효성첨단소재
효성티앤씨
후성
효성
오뚜기
한샘
코오롱인더
동원시스템즈
현대백화점
현대위아
대웅
롯데칠성
LS
대웅제약
현대엘리베이
아시아나항공
한전KPS
아이에스동서
두산
LIG넥스원
동국제강
DL
한국앤컴퍼니
영풍
대한전선
태광산업
에스엘
종근당
LS ELECTRIC
신풍제약
롯데관광개발
현대두산인프라코어
금호타이어
PI첨단소재
명신산업
LX인터내셔널
SK네트웍스
녹십자홀딩스
대한유화
지누스
HDC현대산업개발
신세계인터내셔날
CJ CGV
한세실업

In [21]:
dv['se'].str.contains('현금배당수익률')

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7      True
8      True
9     False
10    False
11    False
12    False
13    False
14    False
Name: se, dtype: bool

In [42]:
dart.report('삼성전자', '배당', 2020)

,rcept_no,corp_cls,corp_code,corp_name,se,thstrm,frmtrm,lwfr,stock_knd
0,20210309000744,Y,00126380,삼성전자,주당액면가액(원),100,100,100,NaN
1,20210309000744,Y,00126380,삼성전자,(연결)당기순이익(백만원),"26,090,846","21,505,054","43,890,877",NaN
2,20210309000744,Y,00126380,삼성전자,(별도)당기순이익(백만원),"15,615,018","15,353,323","32,815,127",NaN
3,20210309000744,Y,00126380,삼성전자,(연결)주당순이익(원),"3,841","3,166","6,461",NaN
4,20210309000744,Y,00126380,삼성전자,현금배당금총액(백만원),"20,338,075","9,619,243","9,619,243",NaN
5,20210309000744,Y,00126380,삼성전자,주식배당금총액(백만원),-,-,-,NaN
6,20210309000744,Y,00126380,삼성전자,(연결)현금배당성향(%),78.00,44.70,21.90,NaN
7,20210309000744,Y,00126380,삼성전자,현금배당수익률(%),4.00,2.60,3.70,보통주
8,20210309000744,Y,00126380,삼성전자,현금배당수익률(%),4.20,3.10,4.50,우선주
9,20210309000744,Y,00126380,삼성전자,주식배당수익률(%),-,-,-,보통주


In [25]:
#from pykrx import stock
#from pykrx import bond
d =stock.stock.get_market_ohlcv("20220214")

AttributeError: module 'pykrx.stock' has no attribute 'stock'